# PyMoq

This is a recreational attempt at replicating parts of the c# moq library.

In [2]:
from typing import Protocol

class IWeb(Protocol):
    def get(url: str) -> str:
        pass

## Goals

- Create a mock based from an interface. It should throw an error if a function is called on it that doesn't exist on the interface

    `m = Mock(IWeb)`

    `m.func() --> throws an error that func doesn't exist on m`

    `m.get() --> throws an error that function signature does not match`
    
- Set return types based on call signature. E.g.

    `m.get.setup().returns("default for all calls")` or

    `m.get.setup("specific str").returns("for that specific string")` or

    `m.get.setup(str).returns("for any string")` or

    `m.get.setup(ConditionType(str, lambda s: s.starts_with("a"))).returns("for a string that matches the condition")`

- Validate calls based on signature. E.g.

    `m.get.validate("specific str").times(1)` or

    `m.get.validate(str).times(0)` or

    `m.get.validate(AnyType(str, lambda s: s.starts_with("a"))).times(2)`

## Faking attributes

### Reading attributes

Listing all attributes is easy:

In [2]:
dir(IWeb)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_is_protocol',
 '_is_runtime_protocol',
 'get']

These have to be filtered, we only need the 'public' attributes. By convention, these are the ones not starting with an underscore.

In [3]:
from typing import Any, List

def is_public(name: str) -> bool:
    return not name.startswith('_')

def get_public_attributes(cls : Any) -> List[str]:
    return list(filter(is_public, dir(cls)))

In [4]:
get_public_attributes(IWeb)

['get']

### Get Function by name

The string name of the functions from above can easily be used to access the corresponding function objects of a class:

In [5]:
getattr(IWeb, 'get')

<function __main__.IWeb.get(url: str) -> str>

### Signature validation

The inspect module gives easy access to the signature of a callable:

In [6]:
from inspect import signature

sig = signature(IWeb.get)
sig

<Signature (url: str) -> str>

We can also check if a given argument list is enough to call a function with the given signature:

In [10]:
sig.bind('url')

<BoundArguments (url='url')>

In [11]:
try:
    sig.bind()
except Exception as e:
    print(e)

missing a required argument: 'url'


From this, we can create a wrapper that checks for the validity of a call, then does some optional pattern matching on the given arguments.

In [186]:
from typing import Callable
import sys

class FunctionMocker:
    def __init__(self, target_function: Callable[[Any], Any], top_level: str=""):
        self.name = target_function.__name__
        self.target_signature = signature(target_function)
        self._error_prefix = f'{top_level}.' if top_level else ''
        
    def _validate_signature(self, *args, **kwargs):
        try:
            self.target_signature.bind(*args, **kwargs)
        except TypeError as e:
            ei = sys.exc_info()
            raise ei[0](TypeError(f'{self._error_prefix}{self.name}{self.target_signature}: {e}')).with_traceback(ei[2].tb_next.tb_next.tb_next) from None
            #raise ei[0], TypeError(f'{self._error_prefix}{self.name}{self.target_signature}: {e}'), ei[2].tb_next
            #raise TypeError(f'{self._error_prefix}{self.name}{self.target_signature}: {e}') from None
        
    def __call__(self, *args, **kwargs):
        self._validate_signature(*args, **kwargs)
        
        print('Correct signature, start pattern matching')
        
    def setup(self):
        pass
        
    def __str__(self):
        return f'Moq: {self.name}{self.target_signature}'
    
    def __repr__(self):
        return str(self)

In [148]:
f = FunctionMocker(IWeb.get)
f('url')

Correct signature, start pattern matching


In [149]:
try:
    f()
except TypeError as e:
    print(e)

get(url: str) -> str: missing a required argument: 'url'


### A first simple Mocker

With the `FunctionMocker` from above we can implement a simple version of a class-specific Mocker.

In [159]:
from typing import TypeVar, Type
T = TypeVar('T')

class Moq:
    def __init__(self, cls: Type[T]) -> T:
        self.cls_name = cls.__name__
        self._extract_attributes(cls)
        
    def _extract_attributes(self, cls):
        self.attribute_names = get_public_attributes(cls)
        self.moq_functions = {name: FunctionMocker(getattr(cls, name), self.cls_name)
                              for name in self.attribute_names}
        
    def __getattr__(self, name):
        if name not in self.moq_functions:
            raise AttributeError(f"No attribute with name '{name}' in class {self.cls_name}")
            
        return self.moq_functions[name]
    
    def __dir__(self):
        return dir(super()) + self.attribute_names

In [160]:
m = Moq(IWeb)

In [161]:
def function_to_test():
    a = 1
    m.get()
    return a+2

Calling a function with an incorrect argument:

In [162]:
try:
    function_to_test()
except Exception as e:
    print(e)

IWeb.get(url: str) -> str: missing a required argument: 'url'


Calling a not existing function:

In [163]:
try:
    m.not_found
except AttributeError as e:
    print(e)

No attribute with name 'not_found' in class IWeb


In [187]:
import types

In [188]:
names = ['get']

In [189]:
a = type('A', (), {name: FunctionMocker(IWeb.get) for name in names})

In [190]:
b = a()

In [191]:
type(b)

__main__.A

In [193]:
b.get.setup

<bound method FunctionMocker.setup of Moq: get(url: str) -> str>